<a href="https://colab.research.google.com/github/chh172/hpjy-gambling-simulation/blob/main/max_cut_in_Pointer%26Actor_Critic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
import random 
from networkx import *
n = 6
p = 0.5
g = erdos_renyi_graph(n, p)
print(g. nodes)
# [0, 1, 2, 3, 4, 5] 
print(g. edges)
# [(0, 1), (0, 2), (0, 4), (1, 2), (1, 5), (3, 4), (4, 5)]

[0, 1, 2, 3, 4, 5]
[(0, 1), (0, 3), (0, 4), (0, 5), (1, 4), (2, 4), (3, 4), (4, 5)]


In [138]:
# Description: this function randomly generate a sequence of graphs G(V,E)
# where graphs are represented by their adjacency matrix A
# and the distribution of V and E are all uniform.
# It takes in 'number' of graphs, 'infimum' and 'supremum' of |V|. 
import numpy as np
from networkx import *
import random
def graph_seq_generator(num, inf, sup):
  atlas = []
  for i in range(num):
    A = adjacency_matrix(erdos_renyi_graph(np.random.randint(inf,sup), 0.5))
    #print(A.todense())
    atlas.append(A.todense())
  return atlas    



In [150]:
import numpy as np
# Description: this function tensor-ize the atlas, preparing for LSTM
def input_generator(atlas):
  return tf.cast(np.array(random.sample(atlas,1)),tf.int64)

In [124]:
# Unit testing
atlas = graph_seq_generator(4,1,5)
#print(atlas[1])
#inp = Input(atlas[1])
#print(inp)
#for i in range(3):
  #print(g[i].nodes)
  #
  #print(g[i].edges)
  #
  #print(len(g[i].nodes))
g = random.sample(atlas,2)
#print(g[1])
input = input_generator(atlas)
print(input)
print(input.shape[0])
print(input.shape[1])
print(input.shape[2])
print(input.shape)
#print(input.shape)
x = Input(shape=(32,))
#x_1 = Input(input)
print(x)


[[[0 1 1 0 1]
  [1 0 0 1 0]
  [1 0 0 1 0]
  [0 1 1 0 1]
  [1 0 0 1 0]]]
1
5
5
(1, 5, 5)
KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name='input_57'), name='input_57', description="created by layer 'input_57'")


In [125]:
import tensorflow as tf
from tensorflow.keras import *
tf.random.set_seed(5)
encoder_inputs = tf.random.normal([1,2, 2]) 

encoder = tf.keras.layers.LSTM(4, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
# TODO: add pointer network layer
decoder = tf.keras.layers.LSTM(4, return_sequences=True, return_state=True)
decoder_outputs, de_state_h, de_state_c = decoder(encoder_inputs)
encoder_states = [de_state_h, de_state_c]
print(encoder_inputs)
print("hello")
print(encoder_states)



tf.Tensor(
[[[-0.18030666 -0.95028627]
  [-0.03964049 -0.7425406 ]]], shape=(1, 2, 2), dtype=float32)
hello
[<tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[-0.15152912, -0.03594875, -0.08885209, -0.08949389]],
      dtype=float32)>, <tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[-0.38909298, -0.0667583 , -0.15113086, -0.1573536 ]],
      dtype=float32)>]


In [ ]:
# training main
import tensorflow as tf
from tensorflow.keras import *
atlas = graph_seq_generator(100,10,50)
# actor/policy works as encoder
actor = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
# pointer works as decoder
pointer = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
# critic/value works as encoder
critic = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
for i in range(100):
  graph = input_generator(atlas)
  print(graph)
  actor_outputs, state_h, state_c = actor(graph)
  decoder_outputs, de_state_h, de_state_c = pointer(actor_outputs)
  value, critic_state_h, critic_state_c = critic(decoder_outputs)

In [153]:
# (inference template) main
import tensorflow as tf
from tensorflow.keras import *
atlas = graph_seq_generator(10,10,50)
graph = input_generator(atlas)
print(graph)
# actor/policy works as encoder
actor = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
# pointer works as decoder
pointer = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
# critic/value works as encoder
critic = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
actor_outputs, state_h, state_c = actor(graph)
decoder_outputs, de_state_h, de_state_c = pointer(actor_outputs)
value, critic_state_h, critic_state_c = critic(decoder_outputs)
print(value)

tf.Tensor(
[[[0 0 1 1 1 1 1 0 0 1 1 0]
  [0 0 1 0 0 1 1 0 1 1 1 1]
  [1 1 0 0 0 1 1 0 1 0 1 0]
  [1 0 0 0 1 0 0 1 0 0 1 0]
  [1 0 0 1 0 1 0 1 1 1 1 1]
  [1 1 1 0 1 0 0 0 0 0 0 1]
  [1 1 1 0 0 0 0 0 0 0 0 0]
  [0 0 0 1 1 0 0 0 1 0 1 0]
  [0 1 1 0 1 0 0 1 0 1 1 1]
  [1 1 0 0 1 0 0 0 1 0 1 0]
  [1 1 1 1 1 0 0 1 1 1 0 0]
  [0 1 0 0 1 1 0 0 1 0 0 0]]], shape=(1, 12, 12), dtype=int64)


InvalidArgumentError: ignored